In [1]:
Dataset_url = "https://www.kaggle.com/datasets/mahyeks/almond-varieties"

In [2]:
# Upload Kaggle json file
!mkdir ~/.kaggle/
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle datasets download -d mahyeks/almond-varieties

Dataset URL: https://www.kaggle.com/datasets/mahyeks/almond-varieties
License(s): CC0-1.0
100% 96.5M/96.5M [00:02<00:00, 27.6MB/s]
100% 96.5M/96.5M [00:02<00:00, 46.4MB/s]


In [4]:
!unzip -q almond-varieties.zip

In [5]:
# Import files
import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision import datasets, transforms, models
from PIL import Image
from tqdm.auto import tqdm
import time

In [6]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
device

'cpu'

In [7]:
# Define transformation for data augmentation or preprocessing
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(degrees=15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [8]:
class CustomImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.samples = []

        # Gather file paths of all images in the folders and their corresponding labels
        for label, folder in enumerate(os.listdir(root_dir)):
            folder_path = os.path.join(root_dir, folder)
            if os.path.isdir(folder_path):
                for file in os.listdir(folder_path):
                  if file.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.webp')):
                    self.samples.append((os.path.join(folder_path, file), label))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, index):
        try:
            image_path, label = self.samples[index]
            image = Image.open(image_path).convert('RGB')

            if self.transform:
                image = self.transform(image)

            return image, label

        except Exception as e:
            # If there's an error opening or processing the image, print the error and return None
            #print(f"Error loading image: {image_path} - {e}")
            return self.__getitem__(index + 1)

In [9]:
# Instantiate custom dataset
root_dir = '/content/dataset'
custom_dataset = CustomImageDataset(root_dir, transform=transform)

In [10]:
len(custom_dataset)

1556

In [11]:
sample_image, sample_label = custom_dataset[0]
print(sample_image.shape)  # Check the shape of the image tensor
print(sample_label)  # Check the label of the image

torch.Size([3, 224, 224])
0


In [12]:
# Define the size of the training set
train_size = int(0.8 * len(custom_dataset))
test_size = len(custom_dataset) - train_size

In [13]:
# Split the dataset into training and testing sets
train_dataset, test_dataset = torch.utils.data.random_split(custom_dataset, [train_size, test_size])

In [14]:
len(train_dataset), len(test_dataset)

(1244, 312)

In [15]:
# Define batch size for DataLoader
batch_size = 16

# Create DataLoader for training set
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Create DataLoader for testing set
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [16]:
model = models.resnet18(pretrained = True)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 120MB/s]


In [17]:
tot_fc = model.fc.in_features
tot_fc

512

In [18]:
# 512 meaning end layer has output of 512

In [19]:
model.fc = nn.Sequential(
    nn.Linear(tot_fc,4),
)
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [20]:
# Setup loss and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model.parameters(),
                             lr=0.01)

In [21]:
def train(dataloader, model, loss_fn, optimizer,device):
    size = len(dataloader.dataset)
    model.to(device)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [22]:
def test(dataloader, model, loss_fn, device):
    size = len(dataloader.dataset)
    model.to(device)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [23]:
start_time = time.time()


# Train and test model
epochs = 5
for epoch in tqdm(range(epochs)):
    print(f"Epoch: {epoch}\n---------")
    train(train_loader, model, loss_fn, optimizer,device)
    test(test_loader, model, loss_fn,device)

current_time = time.time()
total = current_time - start_time
print("Done!")
print(f'Training Took: {total/60} minutes!')

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 0
---------
loss: 1.895216  [   16/ 1244]
Test Error: 
 Accuracy: 79.5%, Avg loss: 0.567511 

Epoch: 1
---------
loss: 0.432339  [   16/ 1244]
Test Error: 
 Accuracy: 86.9%, Avg loss: 0.380547 

Epoch: 2
---------
loss: 1.168617  [   16/ 1244]
Test Error: 
 Accuracy: 84.3%, Avg loss: 0.361527 

Epoch: 3
---------
loss: 0.421009  [   16/ 1244]
Test Error: 
 Accuracy: 87.2%, Avg loss: 0.329100 

Epoch: 4
---------
loss: 0.128737  [   16/ 1244]
Test Error: 
 Accuracy: 90.4%, Avg loss: 0.276196 

Done!
Training Took: 29.76998658577601 minutes!
